In [1]:
#Todo: get data from current API's and from my DB? Probably will add an API for the json data
#So I can get that too, since that will be the best way to test stuff.
# store it into mongo
# then we change this to pull from mongo instead of pulling from.. a CSV file

# then we run some stats, and that's it.

In [42]:
import requests
import json

In [3]:
#import numpy as np
import pandas as pd
import pymongo
#import pymysql
#import matplotlib
#import matplotlib.pyplot as plt
#from sklearn import linear_model``

In [35]:
my_user_data = requests.get("http://paulzplace.asuscomm.com/api/get_user_data?name=spaynkee")

In [46]:
stuff = json.loads(my_user_data.text)
stuff

[{'id': 12179,
  'match_id': 3596031587,
  'role': 'SUPPORT',
  'champion': '117',
  'champion_name': 'Lulu',
  'enemy_champion': '-1',
  'enemy_champion_name': 'None',
  'first_blood': '0',
  'first_blood_assist': '0',
  'kills': '2',
  'deaths': '8',
  'assists': '33',
  'damage_to_champs': '11867',
  'damage_to_turrets': '1514',
  'gold_per_minute': '271.3',
  'creeps_per_minute': '0.25',
  'xp_per_minute': '452.4',
  'wards_placed': '54',
  'vision_wards_bought': '7',
  'wards_killed': '7',
  'items': "Shard of True Ice, Locket of the Iron Solari, Kindlegem, Knight's Vow, Ardent Censer, Boots of Mobility, Oracle Lens",
  'perks': '8214, 8226, 8210, 8237, 8345, 8347',
  'win': 'Fail'},
 {'id': 12180,
  'match_id': 3595971592,
  'role': 'BOTTOM',
  'champion': '29',
  'champion_name': 'Twitch',
  'enemy_champion': '21',
  'enemy_champion_name': 'Miss Fortune',
  'first_blood': '0',
  'first_blood_assist': '0',
  'kills': '9',
  'deaths': '9',
  'assists': '10',
  'damage_to_champs': 

In [55]:
conn = pymongo.MongoClient()
db = conn.lol
collection = db.loldata

In [53]:
collection = collection.drop()
#collection

In [56]:
collection.insert_many(stuff)

In [66]:
things = collection.find_one()


In [67]:
things

{'_id': ObjectId('5f93897663ccc1fc962ed5d9'),
 'id': 12179,
 'match_id': 3596031587,
 'role': 'SUPPORT',
 'champion': '117',
 'champion_name': 'Lulu',
 'enemy_champion': '-1',
 'enemy_champion_name': 'None',
 'first_blood': '0',
 'first_blood_assist': '0',
 'kills': '2',
 'deaths': '8',
 'assists': '33',
 'damage_to_champs': '11867',
 'damage_to_turrets': '1514',
 'gold_per_minute': '271.3',
 'creeps_per_minute': '0.25',
 'xp_per_minute': '452.4',
 'wards_placed': '54',
 'vision_wards_bought': '7',
 'wards_killed': '7',
 'items': "Shard of True Ice, Locket of the Iron Solari, Kindlegem, Knight's Vow, Ardent Censer, Boots of Mobility, Oracle Lens",
 'perks': '8214, 8226, 8210, 8237, 8345, 8347',
 'win': 'Fail'}

In [1]:
def merge_matches_wins(player_matches, matches):
    combined_matches = player_matches.merge(matches, left_on='match_id', right_on='match_id')
    combined_matches = combined_matches.drop(columns=["first_blood_y"])
    combined_matches = combined_matches.rename(columns={"match_id_x": "match_id"})
    combined_matches = combined_matches.rename(columns={"first_blood_x": "first_blood"})
    return combined_matches

In [2]:
# might need to change this to ignore non-recent champs
def get_pos_win_rate(player_matches):
    df = pd.DataFrame(columns=['role', 'games', 'win_rate', 'kda', 'gpm', 'cspm', 'tower_damage', 'champion_damage', 'wards_placed', 'vision_wards_bought', 'wards_killed'])                           
                               
    role_list = ['top', 'jungle', 'middle', 'bottom', 'support']
    
    for role in role_list:
        games_role = player_matches[player_matches['role'] == role]
        role_wins = games_role[games_role['win'] == "Win"].shape[0]
        role_fails = games_role[games_role['win'] == "Fail"].shape[0]
        if role_wins > 0 or role_fails > 0:
            total_games = role_wins+role_fails
            win_ratio = role_wins / total_games
            gpm = games_role['gold_per_minute'].mean()
            cspm = games_role['creeps_per_minute'].mean()
            tower_dmg = games_role['damage_to_turrets'].mean()
            champ_dmg = games_role['damage_to_champions'].mean()
            wards_placed = games_role['wards_placed'].mean()
            vision_bought = games_role['vision_wards_bought'].mean()
            wards_killed = games_role['wards_killed'].mean()
            kda = (games_role['kills'].mean() + games_role['assists'].mean()) / games_role['deaths'].mean()


            # gpm and cspm
            # tower_damage and champ_damage

            df.loc[len(df)] = [role, total_games, win_ratio, kda, gpm, cspm, tower_dmg, champ_dmg, wards_placed, vision_bought, wards_killed]

    return df

In [10]:
def get_performance_for_champ_role(player_matches):
    
    df = pd.DataFrame(columns=['champion', 'role', 'games', 'win_rate', 'kda', 'gpm', 'cspm', 'tower_damage', 'champion_damage', 'wards_placed', 'vision_wards_bought', 'wards_killed'])
                               
    champ_list = list(player_matches['champion_name'].value_counts().index)
                               
    role_list = ['top', 'jungle', 'middle', 'bottom', 'support']
    
    for champ in champ_list:
        player_champ_matches = player_matches[player_matches['champion_name'] == champ]
        for role in role_list:
            champ_games_role = player_champ_matches[player_champ_matches['role'] == role]
            champ_wins = champ_games_role[champ_games_role['win'] == "Win"].shape[0]
            champ_fails = champ_games_role[champ_games_role['win'] == "Fail"].shape[0]
            if champ_wins > 0 or champ_fails > 0:
                total_games = champ_wins+champ_fails
                win_ratio = champ_wins / total_games
                gpm = champ_games_role['gold_per_minute'].mean()
                cspm = champ_games_role['creeps_per_minute'].mean()
                tower_dmg = champ_games_role['damage_to_turrets'].mean()
                champ_dmg = champ_games_role['damage_to_champions'].mean()
                wards_placed = champ_games_role['wards_placed'].mean()
                vision_bought = champ_games_role['vision_wards_bought'].mean()
                wards_killed = champ_games_role['wards_killed'].mean()
                if champ_games_role['deaths'].mean() != 0:
                    kda = (champ_games_role['kills'].mean() + champ_games_role['assists'].mean()) / champ_games_role['deaths'].mean()
                else:
                    kda = (champ_games_role['kills'].mean() + champ_games_role['assists'].mean())
                # gpm and cspm
                # tower_damage and champ_damage

                df.loc[len(df)] = [champ, role, total_games, win_ratio, kda, gpm, cspm, tower_dmg, champ_dmg, wards_placed, vision_bought, wards_killed]
    return df

In [3]:
# In this cell we will be pulling from our APIs, and storing into mongo.

NameError: name 'pd' is not defined

In [ ]:
# This will change to be reading from mongo instead.
matches = pd.read_csv("matches.csv", index_col=0)

In [12]:
spaynkee = pd.read_csv("spaynkee.csv", index_col=0)

In [13]:
dumat = pd.read_csv("dumat.csv", index_col=0)

In [14]:
archemlit = pd.read_csv('archemlit.csv', index_col=0)

In [15]:
a_matches = merge_matches_wins(archemlit, matches)

In [16]:
d_matches = merge_matches_wins(dumat, matches)

In [17]:
s_matches = merge_matches_wins(spaynkee, matches)

In [18]:
sda_matches = matches[~(matches['participants'].str.contains("stylus_crude"))]

In [19]:
sda_matches = sda_matches[~sda_matches['participants'].str.contains("dantheninja6156")]

In [20]:
sda_matches = sda_matches[~sda_matches['participants'].str.contains("csqward")]

In [21]:
sda_matches_backup = sda_matches 

In [22]:
# get all matches where there are 3 participants after dropping all other 3 participants.
sda_matches = sda_matches_backup[sda_matches_backup['participants'].str.contains(",.*,", regex=True)]

In [23]:
sda_matches['win'].value_counts()

Win     134
Fail    118
Name: win, dtype: int64

In [24]:
# before getting performance for role, select new matches only?
s_recent = s_matches.sort_values('start_time', ascending=False)
s_recent = s_matches.head(100)

a_recent = a_matches.sort_values('start_time', ascending=False)
a_recent = a_matches.head(100)

d_recent = d_matches.sort_values('start_time', ascending=False)
d_recent = d_matches.head(500)
#s_recent

In [25]:
s_perf = get_performance_for_champ_role(s_matches)

In [26]:
a_perf = get_performance_for_champ_role(a_recent)

In [27]:
d_perf = get_performance_for_champ_role(d_matches)

In [28]:
s_role = get_pos_win_rate(s_matches)
s_role

,role,games,win_rate,kda,gpm,cspm,tower_damage,champion_damage,wards_placed,vision_wards_bought,wards_killed
0,top,522,0.545977,2.828937,343.017241,5.519157,2924.862069,21510.379310,13.626437,1.250958,1.875479
1,jungle,511,0.489237,2.663353,334.342466,1.215264,776.005871,16016.270059,13.757339,1.778865,2.972603
2,middle,210,0.509524,2.944685,333.900000,5.042857,1658.514286,24444.914286,15.733333,1.580952,2.361905
3,bottom,507,0.546351,2.880061,374.469428,6.258383,2986.757396,23063.708087,12.917160,1.045365,2.276134
4,support,424,0.514151,3.033204,269.955189,0.969340,350.764151,11873.615566,27.971698,2.403302,3.792453


In [29]:
a_role = get_pos_win_rate(a_recent)
a_role

,role,games,win_rate,kda,gpm,cspm,tower_damage,champion_damage,wards_placed,vision_wards_bought,wards_killed
0,top,8,0.500000,2.964286,316.125000,5.250000,2451.875000,17449.250000,9.625000,1.375000,2.500000
1,jungle,4,0.750000,2.333333,335.000000,4.000000,1356.250000,16742.500000,11.750000,1.250000,1.250000
2,middle,68,0.455882,2.308642,325.470588,5.279412,1752.044118,22546.250000,13.441176,1.661765,2.235294
3,bottom,17,0.705882,2.144231,356.705882,5.941176,3057.647059,21314.588235,13.352941,1.411765,2.411765
4,support,3,1.000000,5.200000,323.666667,2.333333,1222.333333,15849.000000,25.333333,1.666667,3.666667


In [30]:
d_role = get_pos_win_rate(d_matches)
d_role

,role,games,win_rate,kda,gpm,cspm,tower_damage,champion_damage,wards_placed,vision_wards_bought,wards_killed
0,top,212,0.561321,3.487952,332.966981,5.566038,1336.599057,21477.929245,10.452830,0.962264,2.108491
1,jungle,279,0.505376,3.049491,326.767025,1.308244,1137.960573,13667.315412,14.616487,1.695341,2.777778
2,middle,279,0.465950,3.155500,341.157706,5.548387,1771.899642,20427.716846,13.154122,1.347670,2.035842
3,bottom,296,0.469595,3.447412,355.432432,6.283784,1984.016892,19646.067568,11.777027,1.152027,2.246622
4,support,854,0.553864,3.865320,280.929742,1.057377,1119.318501,12026.000000,20.305621,2.761124,4.240047


In [31]:
a_perf_top = a_perf[a_perf['role'] == "top"]
a_perf_jg = a_perf[a_perf['role'] == "jungle"]
a_perf_mid = a_perf[a_perf['role'] == "middle"]
a_perf_bot = a_perf[a_perf['role'] == "bottom"]
a_perf_sup = a_perf[a_perf['role'] == "support"]

In [32]:
s_perf_top = s_perf[s_perf['role'] == "top"]
s_perf_jg = s_perf[s_perf['role'] == "jungle"]
s_perf_mid = s_perf[s_perf['role'] == "middle"]
s_perf_bot = s_perf[s_perf['role'] == "bottom"]
s_perf_sup = s_perf[s_perf['role'] == "support"]

In [33]:
d_perf_top = d_perf[d_perf['role'] == "top"]
d_perf_jg = d_perf[d_perf['role'] == "jungle"]
d_perf_mid = d_perf[d_perf['role'] == "middle"]
d_perf_bot = d_perf[d_perf['role'] == "bottom"]
d_perf_sup = d_perf[d_perf['role'] == "support"]

In [34]:
s_perf_mid

,champion,role,games,win_rate,kda,gpm,cspm,tower_damage,champion_damage,wards_placed,vision_wards_bought,wards_killed
11,Orianna,middle,74,0.554054,3.768546,343.770270,5.594595,2489.256757,24650.270270,12.986486,1.770270,1.729730
13,Karma,middle,2,0.500000,4.555556,338.500000,5.000000,1995.500000,28078.500000,17.000000,1.500000,1.500000
31,Brand,middle,18,0.666667,2.507813,329.388889,2.277778,45.833333,28728.222222,32.611111,2.722222,4.944444
34,Ryze,middle,23,0.478261,2.810219,361.521739,5.478261,814.086957,29961.173913,16.565217,1.217391,2.478261
36,Galio,middle,22,0.409091,2.523438,303.409091,4.909091,1393.954545,18042.954545,11.500000,0.636364,1.409091
39,Lux,middle,14,0.642857,3.061728,312.357143,4.214286,988.500000,20759.214286,17.571429,1.285714,2.285714
46,Syndra,middle,15,0.600000,2.772727,334.466667,6.000000,2191.466667,19879.466667,12.200000,1.933333,1.666667
55,Corki,middle,11,0.454545,2.277778,320.454545,5.272727,2078.090909,24421.818182,14.636364,0.909091,3.272727
78,Ziggs,middle,6,0.333333,3.243243,374.166667,6.666667,2225.166667,36556.166667,15.833333,1.333333,2.166667
80,Irelia,middle,1,0.000000,0.111111,257.000000,5.000000,1417.000000,6540.000000,10.000000,3.000000,1.000000


In [36]:
spaynkee

,match_id,role,champion,champion_name,enemy_champion,enemy_champion_name,first_blood,first_blood_assist,kills,deaths,assists,damage_to_champions,damage_to_turrets,gold_per_minute,creeps_per_minute,wards_placed,vision_wards_bought,wards_killed
0,2797485540,top,240,Kled,114.0,Fiora,0,0,4,5,5,13718,1696,301.0,6.0,8,0,1
1,2797465396,bottom,145,Kai'Sa,236.0,Lucian,0,0,11,7,5,21774,2888,419.0,5.0,10,1,3
2,2797416412,jungle,145,Kai'Sa,NaN,NaN,1,0,7,10,9,18153,509,342.0,1.0,11,2,3
3,2797387583,jungle,164,Camille,107.0,Rengar,0,0,2,1,4,3538,0,213.0,0.0,5,0,0
4,2797377646,bottom,236,Lucian,NaN,NaN,1,0,8,2,7,17347,11560,428.0,7.0,8,0,3
5,2797222295,top,240,Kled,24.0,Jax,0,0,5,5,20,21721,6858,360.0,4.0,16,2,5
6,2797183268,top,240,Kled,39.0,Irelia,0,0,3,12,18,20719,2236,356.0,6.0,13,0,6
7,2797166343,top,240,Kled,NaN,NaN,0,0,5,2,4,10470,4337,399.0,7.0,8,1,1
8,2797141761,top,240,Kled,54.0,Malphite,0,0,5,0,7,11098,3763,338.0,6.0,5,0,0
9,2797114846,top,240,Kled,NaN,NaN,0,0,1,9,5,9038,114,247.0,5.0,9,0,2


In [55]:
s_matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 0 to 2173
Data columns (total 29 columns):
match_id               2174 non-null int64
role                   2174 non-null object
champion               2174 non-null int64
champion_name          2174 non-null object
enemy_champion         1421 non-null float64
enemy_champion_name    1421 non-null object
first_blood            2174 non-null int64
first_blood_assist     2174 non-null int64
kills                  2174 non-null int64
deaths                 2174 non-null int64
assists                2174 non-null int64
damage_to_champions    2174 non-null int64
damage_to_turrets      2174 non-null int64
gold_per_minute        2174 non-null float64
creeps_per_minute      2174 non-null float64
wards_placed           2174 non-null int64
vision_wards_bought    2174 non-null int64
wards_killed           2174 non-null int64
win                    2174 non-null object
participants           2174 non-null object
first_baron           

In [48]:
corr = s_matches.corr()

In [59]:
match_result, match_res_index= s_matches['win'].factorize()

In [62]:
test = s_matches['win'].values.reshape(-1, 1)

In [67]:
s_matches_copy = s_matches

In [68]:
s_matches_copy['win'] = s_matches_copy['win'].astype('category')

In [73]:
s_matches_copy['win'] = match_result